# Twitter data extraction and storing in a dataset

In [1]:
import tweepy
import json
import pandas as pd
from scipy.misc import imread
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib as mpl
import csv
import matplotlib.pyplot as plt

import operator
from textblob import TextBlob
from textblob import Word
from textblob.sentiments import NaiveBayesAnalyzer

# Authentication

In [5]:
consumer_key = 'bb32Oh0hhUDz1IVOSJh5QTa89'
consumer_secret = 'ZkU2kYfFqiyT2dJKtg21QROCgy9oMPzgCsmjbyZNB9yQAEDtcn'
access_token = '1069450930045812736-zTW0GusPltHRX8ljuLQXsMgicS7xpi'
access_token_secret = 'tvdh7qXEBcWG5lPyri0jCrfaz0Yji555Gqv7SOIoT71Mt'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret) #Interacting with twitter's API
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API (auth) #creating the API object


#Extracting Tweets
results = []
for tweet in tweepy.Cursor (api.search, q = 'millennials', lang = "en").items(2000): 
    results.append(tweet)
    
print (type(results))
print (len(results))

<class 'list'>
2000


In [6]:
def tweets_df(results):
    id_list = [tweet.id for tweet  in results]
    data_set = pd.DataFrame(id_list, columns = ["id"])
    
    data_set["text"] = [tweet.text for tweet in results]
    data_set["created_at"] = [tweet.created_at for tweet in results]
    data_set["retweet_count"] = [tweet.retweet_count for tweet in results]
    data_set["user_screen_name"] = [tweet.author.screen_name for tweet in results]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in results]
    data_set["user_location"] = [tweet.author.location for tweet in results]
    data_set["Hashtags"] = [tweet.entities.get('hashtags') for tweet in results]
    
    return data_set
data_set = tweets_df(results)

# Finding the things which were asked

In [11]:
data_set


,id,text,created_at,retweet_count,user_screen_name,user_followers_count,user_location,Hashtags
0,1102111172034097158,@newscientist Oh dear... not good. Singaporean...,2019-03-03 07:39:14,0,LotusB10,2657,Gaia,[]
1,1102111162534027264,"RT @louisliaw: Before BlackPink and BTS, there...",2019-03-03 07:39:12,8935,nzlf_a,483,,[]
2,1102111117172629504,"RT @louisliaw: Before BlackPink and BTS, there...",2019-03-03 07:39:01,8935,AimiAsyraaf,441,Selangor,[]
3,1102111114568126464,What this Magogo and Mkhulus know about issues...,2019-03-03 07:39:00,0,LindokuhleSays,418,Johannesburg,[]
4,1102111000000552961,"RT @louisliaw: Before BlackPink and BTS, there...",2019-03-03 07:38:33,8935,nooreffaaa,705,Kota Kinabalu,[]
5,1102110863824056320,#Millennials And Gen Z Workers In 2016 https:/...,2019-03-03 07:38:00,0,46pakuqp,116,,"[{'text': 'Millennials', 'indices': [0, 12]}, ..."
6,1102110689026441216,"RT @louisliaw: Before BlackPink and BTS, there...",2019-03-03 07:37:19,8935,Imnhzq_,543,,[]
7,1102110666113003520,"RT @louisliaw: Before BlackPink and BTS, there...",2019-03-03 07:37:13,8935,syasyushisyu,280,Malas lah nak cakap katmana,[]
8,1102110519564087297,@OfficialZappyK HEY DIPSHIT. SAYING “AVOCADOS ...,2019-03-03 07:36:38,0,dewshberg,0,,[]
9,1102110222682746880,RT @mvollmer1: RT @ipfconline1 What are the #S...,2019-03-03 07:35:27,2,Raychan_16,676,Tokyo│சிங்கப்பூர் குடியரசு│MA,"[{'text': 'SocialMedia', 'indices': [44, 56]},..."


In [42]:
print(len(results))    #1.total number of tweets
l=[]
for i in data_set['user_screen_name']:
    l.append(i)
s= list(set(l)) #2.unique users
f=[] #count of tweets per unique user
for d in s:
    e=l.count(d)
    f.append(e)

Z = [X for _,X in sorted(zip(f,s),reverse=True)]
print(Z[0:10]) #3.top users with maximum tweets
    
    
    
print(len(s))    #unique users

2000
['boomerpanicbot', 'WhyMillennials', 'michellebullet1', 'millenialsbot', 'millennials_bot', 'Bad_Millennials', 'millennial_kill', 'NewInsightWorld', 'ericlo', 'baoguiw']
1847


In [43]:
m=[]  #list of retweet count
for k in data_set['retweet_count']:
    m.append(k)
n=[]    
h=[]
for j in range(len(data_set['text'])):
    if data_set['retweet_count'].iloc[j]==max(m):
        n.append(data_set['text'])
        h.append(data_set['user_screen_name'])
        
print(len(n)) #4.the number of tweets with maximum retweets
print(h) #5.users with maximum retweets

13
[0              LotusB10
1                nzlf_a
2           AimiAsyraaf
3        LindokuhleSays
4            nooreffaaa
5              46pakuqp
6               Imnhzq_
7          syasyushisyu
8             dewshberg
9            Raychan_16
10      BernieUpstateNY
11          nrafifah___
12              SOymmmh
13         thehassanian
14                iiyoj
15           MissLadyMo
16           PlanetNoar
17             shihtzzu
18        HHypocrisy101
19       DerekEberhardt
20            daisyamos
21           adrianafi_
22            _Okashon_
23            Tutoanana
24         aaron_clarey
25         kelsibryanne
26      ArmenteroDorian
27            hkalrnzam
28          nrlnadiah15
29          Lala_Kenosi
             ...       
1970           randykjo
1971          alvinebks
1972       ALIZARIN_ADA
1973             khdjhy
1974      MattSekellick
1975     thinkermachine
1976    supertoughcrybb
1977        arraazriana
1978         shahirfzli
1979           dsmyrnos
1980        

In [33]:
q=[]
o= list(set(m))
for p in o:
    a= m.count(p)
    q.append(a)
print(max(q))

for r in range(len(m)):
    if m.count(m[r])==max(q):
        print(data_set['text'].iloc[r])  #6.tweets with the most favourite count of retweets
        

688
@newscientist Oh dear... not good. Singaporeans are 2nd most sleep deprived people @62% not getting enough sleep. 1… https://t.co/ugXXWoNYVh
What this Magogo and Mkhulus know about issues relating to millennials. They are a millennial old https://t.co/W8fMm6mwuP
#Millennials And Gen Z Workers In 2016 https://t.co/uOnsAWSXK4  via Forbes #smartcities #SmartCity #lifestyle https://t.co/Om9VRqzRLZ
@OfficialZappyK HEY DIPSHIT. SAYING “AVOCADOS ARE JUST PEARS FOR MILLENNIALS” IS BULLSHIT AND YOU KNOW IT.
I thought hate and tribalism was dying in Nigeria, but the millennials have graduated this hate and tribalism to a… https://t.co/85r9mR0GVv
I get a kick out of these millennials that are obsessed with the 90’s. Half of them wouldn’t last a day in that dec… https://t.co/1dfZXlATS5
Trying to decide how to present himself as a ‘cool &amp; hip’ Metallica dude?  Many millennials might ask:  what’s Meta… https://t.co/0qjNZFcMas
You dipshit #Millennials believe in.#GlobalWarming .  here's an ar

(Using the Howe Scale)
Nice of Tywin to tell late 30s Jaime he still has his youth. Makes so much of us older Millennials feel, well, some… https://t.co/o1FueJtc3X
Mr Bernanke’s unorthodox “cash for trash” scheme, otherwise known as quantitative easing, drove up asset prices and… https://t.co/KqjMhmJpgP
Help! Millennials Depleted Our Driveway !!!!
Mr Bernanke’s unorthodox “cash for trash” scheme, otherwise known as quantitative easing, drove up asset prices and… https://t.co/paKsU85s2U
Millennials killed the henna commerce.
Why aren't millennials buying sandblasters?
@PlanetTyrus Hey Brother I always enjoy seeing you do your thing. I was watching you tonight on the @GregGutfeld Sh… https://t.co/sIbeC3mzh2
Why aren't Millennials propagating income? 💵
Want this cramped 2-brm house in Glen Eden? Give up social contact and it's yours, Millennials
The thing about traveling with  millennials is the Jello shots.
@Keatronn @HowardSchultz Allows them the opportunity to improve their life.
Mille

In [36]:
u=[] #all locations
for t in data_set['user_location']:
    u.append(t)
v= list(set(u)) #unique locations
x= [] # count of each unique location
for w in v:
    b= u.count(w)
    x.append(b)

for y in range(len(v)):
    print(v[y],x[y]) #7.location and the count of each location
    
    

 584
internet 1
Coming to a town near you 1
England, United Kingdom 1
Melbourne, Victoria, AU 1
Moon 1
Philadelphia 3
Dakhur Province, Bajor 1
Rhode Island 1
GO the national anthem 1
rust belt 1
Taguig City 1
RVA 1
Texas...thankfully a blue part 1
🇲🇾  1
Japan 1
https://justicedemocrats.com/ 1
Berkeley, CA 1
Indonesia 3
Zimbabwe 1
your mother’s butt 1
perlis,malaysia 1
Los Angeles 1
Hamilton, Ontario 1
fangirl in you area 1
Skylar Jai & Kingston Jamar 1
Monroe County, Indiana, USA 1
@bts_twt 1
The PNW 1
Tennessee 1
on my way 1
San Francisco 3
Here, Earth 1
Nashville, TN 1
Gunnedah, New South Wales 1
The internets 1
Every. Where.  1
Gippsland. Australia 1
NY 1
Southampton, NY 1
333 1
  2
Челябинск, Россия 1
Kentucky 2
Yogyakarta, Indonesia 1
Puebla, Puebla 1
Connecticut River Valley, MA 1
Pasadena/Los Angeles CA 1
South Dakota 1
Damansara, Selangor 1
🇲🇾 5
Kansas 1
Chennai 2
ATL ↔️LA🏝 1
PH 2
Eastern Time Zone 1
INA 1
() 1
Seoul 1
St. Louis, Missouri 1
Chicago, USA.  1
Connecticut. East Co

BB BP 1
See My Fool Articles: 1
LAX-HNL 1
Auckwood 1
bki 🛫kul 1
Rio Vista, CA 1
Bermuda 1
Bada Bing 1
Charlottesville, Virginia 1
91109 1
Fort Collins, CO 1
IG: @saabmina 1
Raleigh, NC 1
they/them 1
Worldwide 4
Edge of the six yard box 1
In your heart 1
Shanghai 1
Land of the Bloated Egos 1
Texas, USA  1
Arizona, USA 2
Saint Charles, MO 1
wherever da noodles at 1
Mongolia 1
The Wild West 1
Pharr, TX. 1
Republic of the Philippines 4
 MY FARM OR ON THE H2O in MI 1
Lathrop, CA 1
Antwerp 1
el salvador 1
Fatin Najiha 1
Cleveland 1
under the aileron 1
Boulder, CO 1
Spokane Valley, WA (STATE)  1
Manila City, National Capital  1
Perth, Western Australia 1
Ipoh  1
Lookin for My Glucose Guardian 1
axu 1
The Swamp 1
Louisville, KY 1
MBA Tulane U-Independent voter 1
US | UK | CANADA | AUSTRALIA 2
Port Elizabeth 1
808 1
Toronto 3
Nicaragua 1
Florida  1
stan exo 1
Miramichi, New Brunswick, CAN 1
Newport Beach, CA 1
Perlis 1
Klang Bandar Diraja 1
Arizona 2
New York 4
Born GA. Raised AZ. Stuck TN 1
Th